# Demo

### Setup

In [1]:
# !pip install azure-cognitiveservices-speech

In [2]:
import azure.cognitiveservices.speech as speechsdk

Define `AZURE_SPEECH_KEY`, `AZURE_SPEECH_REGION`, `LANGUAGE`:

In [3]:
exec(open('../instance/config.py').read())

### Speech-to-text

In [4]:
def speech_to_text():
    speech_config = speechsdk.SpeechConfig(subscription=AZURE_SPEECH_KEY, region=AZURE_SPEECH_REGION)
    speech_config.speech_recognition_language = LANGUAGE

    audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    print('Speak into your microphone.')
    speech_recognition_result = speech_recognizer.recognize_once_async().get()

    if speech_recognition_result.reason == speechsdk.ResultReason.RecognizedSpeech:
        return speech_recognition_result.text
    elif speech_recognition_result.reason == speechsdk.ResultReason.NoMatch:
        print('No speech could be recognized: {}'.format(speech_recognition_result.no_match_details))
    elif speech_recognition_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_recognition_result.cancellation_details
        print('Speech Recognition canceled: {}'.format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            print('Error details: {}'.format(cancellation_details.error_details))
            print('Did you set the speech resource key and region values?')

### Text-to-speech

In [5]:
def text_to_speech(text):
    speech_config = speechsdk.SpeechConfig(subscription=AZURE_SPEECH_KEY, region=AZURE_SPEECH_REGION)
    audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)

    speech_config.speech_synthesis_voice_name = 'fr-FR-HenriNeural'

    speech_synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

    speech_synthesis_result = speech_synthesizer.speak_text_async(text).get()

    if speech_synthesis_result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
        print('Speech synthesized for text [{}]'.format(text))
    elif speech_synthesis_result.reason == speechsdk.ResultReason.Canceled:
        cancellation_details = speech_synthesis_result.cancellation_details
        print('Speech synthesis canceled: {}'.format(cancellation_details.reason))
        if cancellation_details.reason == speechsdk.CancellationReason.Error:
            if cancellation_details.error_details:
                print('Error details: {}'.format(cancellation_details.error_details))
                print('Did you set the speech resource key and region values?')

### Text-to-math

In [6]:
def text_to_math(text):
    mapping = {
        'égale': '=',
        'plus': '+',
        'moins': '-',
        'fois': '*',
        'sur': '/',
        'au': '',
        'carré': '^ 2',
        'cube': '^ 3',
        'un': '1',
    }
    
    text = text.translate(str.maketrans('', '', '.')) # Remove the '.' characters
    L = text.split()
    
    for i, elem in enumerate(L):
        if elem in mapping:
            L[i] = mapping[elem]
            
    return ' '.join(filter(lambda x: x, L))

### Demo

In [7]:
equations = []
index = -1

while True:
    command = input('> ').strip()
    if command == '':
        text = speech_to_text()
        math = text_to_math(text)
        text_to_speech(text)
        print(math)
        equations.append(math)
        index = len(equations) - 1
    elif command == 'c': # [C]urrent
        if equations:
            print(equations[index])
    elif command == 'a': # [A]ll
        print(*equations, sep='\n')
    elif command == 'p': # [P]revious
        index = max(0, index - 1)
        if equations:
            print(equations[index])
    elif command == 'n': # [N]ext
        index = min(index + 1, len(equations) - 1)
        if equations:
            print(equations[index])
    elif command == 'e': # [E]dit
        text = speech_to_text()
        math = text_to_math(text)
        text_to_speech(text)
        print(math)
        if equations:
            equations[index] = math
        else:
            equations.append(math)
    elif command == 'q': # [Q]uit
        break
    print()

> 
Speak into your microphone.
Speech synthesized for text [3X+2 égale 5.]
3X+2 = 5

> 
Speak into your microphone.
Speech synthesized for text [3X égale 5-2.]
3X = 5-2

> 
Speak into your microphone.
Speech synthesized for text [3X égale 3.]
3X = 3

> 
Speak into your microphone.
Speech synthesized for text [X égale un.]
X = 1

> e
Speak into your microphone.
Speech synthesized for text [X égale 3 sur 3.]
X = 3 / 3

> 
Speak into your microphone.
Speech synthesized for text [X égale un.]
X = 1

> a
3X+2 = 5
3X = 5-2
3X = 3
X = 3 / 3
X = 1

> q
